# Descripción

[Pagina de buscaminas en el que se baso el algoritmo](https://www.solitar.io/buscaminas)

# Dependencias

Si no puedes correr el codigo, descomenta las siguientes 2 lineas de codigo o ejecutalas directamente desde tu terminal y reinicia el kernel de python.

<strong>Nota</strong>: Al ejecutarse por primera vez el proyecto, se tardara unos minutos

In [ ]:
# !pip install pyautogui opencv-python easyocr

In [ ]:
# !pip install torch==2.3.1 torchaudio==2.3.1 torchvision==0.18.1

# Librerias

In [ ]:
import os
import cv2
import time
import easyocr
import logging
import pyautogui
import numpy as np
from collections import deque
import matplotlib.pyplot as plt

In [ ]:
start_time = time.time()

# Captura de pantalla

In [ ]:
# Obtenemos la imagen de la captura de pantalla
def take_screenshot():
    screenshot = pyautogui.screenshot()
    return cv2.cvtColor(np.array(screenshot), cv2.COLOR_RGB2BGR)

# Obtenemos los colores presentes en el tablero
def get_colors():
    return np.array([
        [247, 236, 227], # White
        [158, 92, 21],   # Blue
        [200, 113, 27],  # Background
    ], dtype=np.uint8)

# Creamos las mascaras de los colores
def create_mask_colors(screenshot, colors):
    return [cv2.inRange(screenshot, color, color) for color in colors]

# Obtenemos los contornos de la imagen
def get_contours(masks):
    combined_mask = np.bitwise_or.reduce(masks)
    contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

# Obtenemos las dimensiones de una imagen (x, y, width, height)
def get_dimensions(contours):
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    return x, y, w, h

In [ ]:
# Captura de pantalla
screenshot = take_screenshot()
colors = get_colors()
masks = create_mask_colors(screenshot, [colors[-1]])
colors = colors[:-1]

# Obtenemos la imagen de la tabla
table_contours = get_contours(masks)
table_dims = get_dimensions(table_contours)

# Obtenemos los colores
x, y, w, h = table_dims
square_masks = create_mask_colors(screenshot[y:y+h, x:x+w], colors)

# Cuadrado blanco
square_contours_white = get_contours([square_masks[0]])
square_dims_white = get_dimensions(square_contours_white)

# Cuadrado azul
square_contours_blue = get_contours([square_masks[1]])
square_dims_blue = get_dimensions(square_contours_blue)

# Dimensiones del cuadrado
index = 0
square_distance = max(square_dims_white[index], square_dims_blue[index])
square_size = max(np.concatenate((square_dims_white[2:], square_dims_blue[2:])))

border_size, line_size = 12, 4
square_dims = (square_size, border_size, line_size)

table_dims, square_dims

In [ ]:
x, y, w, h = table_dims
plt.imshow(cv2.cvtColor(screenshot[y:y+h, x:x+w], cv2.COLOR_BGR2RGB), cmap='gray')
plt.axis('off')  # Ocultar los ejes
plt.show()

# Lectura del tablero

In [ ]:
# Modelo descifrador de texto
logging.getLogger().setLevel(logging.ERROR)
reader = easyocr.Reader(['en'])

In [ ]:
# Obtenemos la imagen de una casilla del tablero
def get_square(screenshot, table_dims, i, j, square_size):
    square_size, border_size, line_size = square_dims
    
    sx = j * (square_size + line_size * int(j > 0)) + border_size
    sy = i * (square_size + line_size * int(i > 0)) + border_size
    
    x, y, w, h = (
        table_dims[0] + sx, 
        table_dims[1] + sy, 
        square_size, 
        square_size
    )
    return screenshot[y:y+h, x:x+w]

# Redimensionamiento de la imagen
def scale_image(square, width, inter=cv2.INTER_AREA):
    original_height, original_width = square.shape[:2]
    if original_width == width:
        return square
    
    ratio = width / float(original_width)
    height = int(original_height * ratio)
    
    return cv2.resize(square, (width, height), interpolation=inter)

# Conversion de string a numero
def str_to_number(str_value):
    try:
        return int(str_value)
    except ValueError:
        return

# Desciframos el numero de la casilla    
def get_number(square):
    results = reader.readtext(square)
    if not results:
        return -1

    sorted_results = sorted(results, key=lambda x: x[2], reverse=True)
    return str_to_number(sorted_results[0][1])

# Obtenemos el valor de una casilla (revelada_con_numero: numero, oculta: 0, revelada_sin_numero: -1)
def get_cell_value(square, color):
    mask = cv2.inRange(square, color, color)
    color_percentage = (np.sum(mask) / (mask.size * 255)) * 100

    if color_percentage == 0:
        return 0

    return get_number(square)

In [ ]:
_, _, width, height = table_dims
square_size, border_size, line_size = square_dims

# Dimensiones del tablero
n = ((height - 2 * border_size + line_size) // (square_size + line_size))
m = ((width - 2 * border_size + line_size) // (square_size + line_size))

n, m

In [ ]:
table = np.zeros((n, m))

for i in range(len(table)):
    for j in range(len(table[i])):
        square = get_square(screenshot, table_dims, i, j, square_size)
        resized_square = scale_image(square, 80)
        table[i, j] = get_cell_value(resized_square, colors[1])
    print(f"Fila {i + 1} de {len(table)} leida")

In [ ]:
# Mostramos el tablero
def show_table(table):
    str_value = ""
    for row in table:
        for col in row:
            num = float(col)
            value = f" {num:.0f}" if num < 0 else f"  {num:.0f}"
            str_value += value + ""
        str_value += "\n"

    print(str_value)

In [ ]:
show_table(table)

# Calculo de heuristica

In [ ]:
# Obtenemos el valor de una celda (-3 si no existe)
def get_cell(table, i, j):
    if 0 <= i < len(table) and 0 <= j < len(table[0]):
        return table[i][j]
    return -3

# Celdas adyacentes a una celda
def get_adjacent_cells(table, i, j):
    adjacent_cells = []
    
    for dx, dy in [(-1, -1), (0, -1), (1, -1), (1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0)]:
        ni, nj = i + dy, j + dx
        cell_value = get_cell(table, ni, nj)
        
        if cell_value != -3:
            adjacent_cells.append((ni, nj, cell_value))
    
    return adjacent_cells

# Celdas con numeros
def adjacent_cell_with_positive_numbers(table, i, j):
    return [(ni, nj, cell_value) for ni, nj, cell_value in get_adjacent_cells(table, i, j) if cell_value > 0]

# Numero de celdas con numeros
def num_adjacent_cell_with_positive_numbers(table, i, j):
    return sum(1 for ni, nj, cell_value in get_adjacent_cells(table, i, j) if cell_value > 0)

# Minas alrededor de una celda
def mines_around_cell(table, i, j):
    return sum(1 for ni, nj, cell_value in get_adjacent_cells(table, i, j) if cell_value == -2)

# Celdas vacias alrededor de una celda
def empty_cells_around(table, i, j):
    return {(ni, nj) for ni, nj, cell_value in get_adjacent_cells(table, i, j) if cell_value == 0}

# Caso en el que se presenta una esquina
# Ejemplo:
# [-1] [-1] [-1]
# [-1]  [1]  [2]
# [-1]  [2]  [x]
# x es la celda a evaluar
def is_special_case(table, i, j):
    cases = [
        [ # Arriba-Derecha
            [(0, -1), (1, 0)], # Diagonales con numeros
            (1, -1) # Diagonal con un 1
        ],
        [ # Derecha-Abajo
            [(1, 0), (0, 1)], # Diagonales con numeros
            (1, 1) # Diagonal con un 1
        ],
        [ # Abajo-Izquierda
            [(0, 1), (-1, 0)], # Diagonales con numeros
            (-1, 1) # Diagonal con un 1
        ],
        [ # Izquierda-Arriba
            [(-1, 0), (0, -1)], # Diagonales con numeros
            (-1, -1) # Diagonal con un 1
        ]
    ]

    if get_cell(table, i, j) == 0: # Si es una celda oculta
        for case in cases:
            is_numbers = all(get_cell(table, i + dy, j + dx) > 0 for dx, dy in case[0]) # Rodeada por 3 celdas con numeros
            
            dx, dy = case[1]
            ni, nj = i + dy, j + dx
            num_empties = empty_cells_around(table, ni, nj)
            cell_value = get_cell(table, ni, nj)
            mines_around = mines_around_cell(table, ni, nj)

            # Si las celdas vacias es igual a las minas restantes para que la celda tenga el numero indicado de minas alrededor
            if num_empties == cell_value - mines_around and is_numbers:
                return True

    return False

# Se pone el caso especial en el tabler y se evalua si es valido o no
def is_valid_special_case(table, i, j):
    if is_special_case(table, i, j):
        new_table = [row.copy() for row in table]
        new_table[i][j] = -2
        return is_valid_board(new_table)
    return False

# Calculo de la heuristica
def heuristic(table, i, j):
    mines_around, empty_cells = 0, set()
    adjacent_cells = adjacent_cell_with_positive_numbers(table, i, j)

    # Si es un caso especial, la heuristica es el mayor numero posible
    if is_valid_special_case(table, i, j):
        return 100
    
    # Si la celda no es oculta o si no tiene celdas adyacentes
    if not adjacent_cells or get_cell(table, i, j) != 0:
        return 0

    # Calculo de minas y celdas vacias alrededor
    for ni, nj, cell_value in adjacent_cells:
        mines_around += cell_value - mines_around_cell(table, ni, nj)
        empty_cells.update(empty_cells_around(table, ni, nj))

    # Si solo hay una celda vacia y aun faltan por colocar minas, significa que la celda actual es esa mina
    if len(empty_cells) == 1 and mines_around > 0:
        return 100
        
    return mines_around / len(empty_cells) if empty_cells else int(mines_around > 0)

# Algoritmo para resolver el buscaminas

In [ ]:
# Calculo de la heuristica para hallar donde se ubica una mina
def calculate_heuristics(table):
    heuristics = []
    for i in range(len(table)):
        for j in range(len(table[i])):
            value = heuristic(table, i, j)
            if value > 0: # La celda es valida para evaluar
                heuristics.append((i, j, value))
    return sorted(heuristics, key=lambda x: x[2])

# Validacion del tablero
def is_valid_board(table, full=False):
    for i in range(len(table)):
        for j in range(len(table[i])):
            cell_value = get_cell(table, i, j)
            mines_around = mines_around_cell(table, i, j)
            adjacent_cells = adjacent_cell_with_positive_numbers(table, i, j)

            # El tablero no es valido cuando:
            # 1. Si no esta lleno y hay alguna celda con numero con mas minas alrededor que lo que indica su numero
            # 2. Si esta lleno y todas las celdas con numeros no tiene la cantidad exacta de minas alrededor
            cond = (cell_value < mines_around) or (mines_around < cell_value and full)
            
            if adjacent_cells and cell_value > 0 and cond:
                return False
    return True

# Se puede continuar colocando minas
def can_continue(table):
    for i in range(len(table)):
        for j in range(len(table[i])):
            cell = get_cell(table, i, j)
            # Se pueden colocar minas si hay alguna celda con numero con menos minas alrededor
            if 0 < cell and mines_around_cell(table, i, j) < cell:
                return True
    return False

# Se toman todas las posibilidades de colocar minas en el tablero y se halla la probabilidad de cada posicion
def calculate_probability(matrices, target_number, row_index, col_index):
    total_matrices = len(matrices)
    count = sum(matrix[row_index][col_index] == target_number for matrix in matrices)
    return 0 if total_matrices == 0 else (count / total_matrices), count

# Se llena una matriz con las probabilidades de que una celda sea una mina
def get_probabilities(table, paths, n, m):
    prob_table = np.ones((n, m))
    
    for i in range(n):
        for j in range(m):
            prob, count = calculate_probability(paths, -2, i, j)
            num_adj = num_adjacent_cell_with_positive_numbers(table, i, j)
            cell = get_cell(table, i, j)
            
            if 0 < count or (cell == 0 and 0 < num_adj):
                prob_table[i, j] = prob
            else:
                prob_table[i, j] = 1
                
    return prob_table

# Se encuentra la posicion (i, j) de la celda con menos probabilidad de que sea una mina
def find_best_movement(table, valid_paths):
    print(len(valid_paths))
    prob_table = get_probabilities(table, valid_paths, len(table), len(table[0]))
    np_matrix = np.array(prob_table)
    return np.unravel_index(np.argmin(prob_table), prob_table.shape)

# Algoritmo para resolver el buscaminas
def minesweeper_solver(table, heuristics_list, valid_paths, max_possibilities, visited_tables=set(), invalid_movements=[]):
    # Si se pasa el numero de posibilidades
    if len(valid_paths) > max_possibilities:
        return
    
    heuristics_list = [heur for heur in heuristics_list if (heur[0], heur[1]) not in invalid_movements]
    
    # Si no se puede continuar o no hay mas movimientos
    if not can_continue(table) or not heuristics_list:
        if is_valid_board(table, full=True): # Si el tablero es valido, se añade la posibilidad
            valid_paths.append(table)
        return
    
    i, j, h = heuristics_list.pop() # Se obtiene la posicion (i, j) con la mayor heuristica
    if h == 100: # Si es un caso especial, se descartan las demas posibilidades
        heuristics_list.clear()
    
    # Se crea un nuevo tablero
    new_table = [row.copy() for row in table]
    new_table[i][j] = -2 # Se asume que la posicion (i, j) es mina
    
    # Si un tablero se repite, no se sigue
    table_tuple = tuple(tuple(row) for row in new_table)
    if table_tuple in visited_tables:
        return
    
    visited_tables.add(table_tuple)
    new_invalid_movements = invalid_movements.copy()
    
    # Se valida si la posicion (i, j) es mina
    if is_valid_board(new_table): # Si es correcto, se continua
        new_heuristics = calculate_heuristics(new_table)
        minesweeper_solver(new_table, new_heuristics, valid_paths, max_possibilities, visited_tables, new_invalid_movements)
    
    else: # Sino, se añade como movimiento invalido
        new_invalid_movements.append((i, j))
    
    # Se intenta con los movimientos restantes
    minesweeper_solver(table, heuristics_list, valid_paths, max_possibilities, visited_tables, new_invalid_movements)

# Solución

In [ ]:
valid_paths, max_possibilities = [], 100
heuristics_list = calculate_heuristics(table)
minesweeper_solver(table, heuristics_list, valid_paths, max_possibilities)

In [ ]:
square_i, square_j = find_best_movement(table, valid_paths)

In [ ]:
# Se muestra el mejor movimiento
def show_best_movement(table_dims, i, j, square_dims):
    x, y, w, h = table_dims
    square_size, border_size, line_size = square_dims
    
    sx = j * (square_size + line_size * int(j > 0)) + border_size
    sy = i * (square_size + line_size * int(i > 0)) + border_size
        
    table = screenshot[y:y+h, x:x+w]    
    cv2.rectangle(table, (sx, sy), (sx+square_size, sy+square_size), (0, 0, 0), 2)

    plt.imshow(cv2.cvtColor(table, cv2.COLOR_BGR2RGB), cmap='gray')
    plt.axis('off')  # Ocultar los ejes
    plt.show()

In [ ]:
show_best_movement(table_dims, square_i, square_j, square_dims)

# Tiempo transcurrido

In [ ]:
end_time = time.time()

In [ ]:
elapsed_time = end_time - start_time

minutes = int(elapsed_time // 60)
seconds = int(elapsed_time % 60)

print(f"{minutes}min {seconds}s")